# 기초상품 추천 코드 구현해보기
주어진 데이터를 활용하여 유저 아이디와 원하는 제품 유형을 입력하면   
해당 유저와 유사도가 높은 유저의 평점 높은 제품을 추천해주는 함수 만들기

* 유저데이터 파일명 : user_data
* 제품데이터 파일명 : product_data

In [1]:
import pandas as pd

#유저데이터 불러오기
u_data = pd.read_csv('user_data.csv')

#제품데이터 불러오기
p_data = pd.read_csv('product_data.csv')

In [2]:
u_data.head()

,user_id,rating,product_id,skin
0,1,1,20,민감성
1,1,5,5,민감성
2,1,2,3,민감성
3,1,4,34,민감성
4,1,2,18,민감성


In [3]:
p_data.tail(10)

,product_id,product_name,product_type,target_skin_type,ingredient,Unnamed: 5,Unnamed: 6
56,57.0,toner_9,토너,복합성,"병풀추출물(39.2%), 약모밀추출물(28.1%), 감초뿌리추출물(25.7%), 1...",NaN,NaN
57,58.0,toner_10,토너,건성,"정제수, 프로판다이올, 변성알코올, 글리세린, 나이아신아마이드*, 1,2-헥산다이올...",NaN,NaN
58,59.0,toner_11,토너,복합성,"정제수, 2,3-부탄다이올, 에톡시다이글라이콜, 글루코노락톤, 1,2-헥산다이올, ...",NaN,NaN
59,60.0,toner_12,토너,복합성,"정제수, 부틸렌글라이콜, 프로판다이올, 1,2-헥산다이올, 나이아신 아마이드, 판테...",NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
p_data = p_data.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1)

In [5]:
p_data = p_data.dropna() # 인덱스 60~65는 필요 없는 데이터이므로 삭제

In [6]:
p_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        60 non-null     float64
 1   product_name      60 non-null     object 
 2   product_type      60 non-null     object 
 3   target_skin_type  60 non-null     object 
 4   ingredient        60 non-null     object 
dtypes: float64(1), object(4)
memory usage: 2.8+ KB


In [7]:
#product_id 기준으로 유저,제품 데이터 합쳐서 df에 저장하고 user_id를 기준으로 오름차순 정렬
df = pd.merge(u_data, p_data, on='product_id')
df.set_index('user_id', inplace=True)
df.head()

,rating,product_id,skin,product_name,product_type,target_skin_type,ingredient
user_id,,,,,,,
1,1,20,민감성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...
3,2,20,민감성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...
4,2,20,건성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...
6,2,20,지성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...
7,4,20,복합성,serum_2,세럼,지성,정제수 변성알코올 글리세린 프로판다이올 나이아신아마이드 락틱애씨드 하이드록시에틸피페...


In [8]:
#각 유저가 제품에 남긴 평점들로 이루어진 데이터 프레임(df_users) 만들기 인덱스:user_id, 컬럼명:product_id
df_users= df.copy().pivot_table('rating', index='user_id', columns='product_id')
df_users.fillna(0, inplace=True)
df_users

product_id,1,2,3,4,5,6,7,8,9,10,...,51,52,53,54,55,56,57,58,59,60
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,2.0,0.0,5.0,0.0,2.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
2,2.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,4.0,3.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,4.0,0.0,0.0,2.0,1.0,0.0,0.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2.0,5.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0


In [9]:
#유저간 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(df_users.values, df_users.values)
cos_matrix

array([[1.        , 0.37756255, 0.11836501, ..., 0.30848211, 0.30876888,
        0.33248992],
       [0.37756255, 1.        , 0.10316111, ..., 0.22016375, 0.30076748,
        0.11435019],
       [0.11836501, 0.10316111, 1.        , ..., 0.3901178 , 0.2384219 ,
        0.27498715],
       ...,
       [0.30848211, 0.22016375, 0.3901178 , ..., 1.        , 0.09955444,
        0.40333349],
       [0.30876888, 0.30076748, 0.2384219 , ..., 0.09955444, 1.        ,
        0.23491702],
       [0.33248992, 0.11435019, 0.27498715, ..., 0.40333349, 0.23491702,
        1.        ]])

In [10]:
cos_matrix.shape

(100, 100)

In [11]:
#위에서 계산한 cos_matrix를 데이터프레임으로 만듭니다. 
df_users_cosine= pd.DataFrame(data=cos_matrix, index=df_users.index, columns=df_users.index)
df_users_cosine.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.377563,0.118365,0.195347,0.465273,0.350255,0.369234,0.382708,0.209271,0.388365,...,0.434633,0.327872,0.595550,0.253139,0.292668,0.306267,0.256333,0.308482,0.308769,0.332490
2,0.377563,1.000000,0.103161,0.106745,0.226767,0.327382,0.163269,0.323855,0.210206,0.190810,...,0.244048,0.212365,0.221078,0.246255,0.231953,0.218187,0.195578,0.220164,0.300767,0.114350
3,0.118365,0.103161,1.000000,0.328517,0.265719,0.423921,0.313412,0.399815,0.202431,0.194410,...,0.362167,0.376298,0.201629,0.211564,0.175430,0.439941,0.336052,0.390118,0.238422,0.274987
4,0.195347,0.106745,0.328517,1.000000,0.306010,0.280929,0.385025,0.287416,0.299863,0.477361,...,0.165149,0.281387,0.224007,0.263200,0.185619,0.296223,0.209036,0.376882,0.164470,0.357317
5,0.465273,0.226767,0.265719,0.306010,1.000000,0.209787,0.217964,0.221134,0.097609,0.239447,...,0.267159,0.319854,0.243056,0.183303,0.338358,0.361589,0.202548,0.051116,0.259230,0.302407


## 입력한 아이디의 유저와 비슷한 유저가 만족한 제품 추천하는 함수 만들기
1. df_users_cosine에서 입력한 아이디와 유사도 높은 5명을 sim_users로 지정
2. df에서 user_id가 sim_users의 인덱스와 일치하는 값들을 sim_user_df에 할당
3. sim_user_df에서 입력한 product_type과 일치하는 값들을 지정하고 rating을 기준으로 내림차순한 결과를 products에 할당
4. 평점 3점인 제품은 1, 평점 4점인 제품은 2, 평점 5점인 제품은 3의 가중치로 계산하여 products에 선호도를 측정한다
5. 가중치가 높은 제품순으로 prod_list에 저장한다
6. prod_list에서 상위 5개의 제품을 출력한다

In [67]:
def user_based_recommend(user_id, product_type):
    # 1번 : df_users_cosine에서 입력한 아이디와 유사도 높은 5명을 sim_users로 지정
    sim_users = df_users_cosine.loc[user_id].sort_values(ascending=False).index[1:6].tolist()
    
    # 2번 : df에서 user_id가 sim_users의 인덱스와 일치하는 값들을 sim_user_df에 할당
    sim_user_df = df.loc[sim_users] # 2번
    
    # 3번 : sim_user_df에서 입력한 product_type과 일치하는 값들을 지정하고 rating을 기준으로 내림차순한 결과를 products에 할당
    products = sim_user_df[sim_user_df['product_type'] == product_type].sort_values('rating', ascending=False)
    
    dic = dict() # 제품명과 가중치를 저장할 빈 딕셔너리를 생성
    
    # 4번 : 평점 3점인 제품은 1, 평점 4점인 제품은 2, 평점 5점인 제품은 3의 가중치로 하여 products에 선호도를 계산한다
    ratings_mapping = {3: 1, 4: 2, 5: 3}
    for rating, group in products.groupby('rating'):
        for name in group['product_name']:
            dic[name] = dic.get(name, 0) + ratings_mapping.get(rating, 0)
    
    prod_list = [] # 가중치가 높은 순으로 제품을 저장할 리스트
    
    # 5번 : 가중치가 높은 제품순으로 prod_list에 저장한다
    sorted_data = sorted(dic.items(), key=lambda x: x[1], reverse=True)
    prod_list = [name for name , _ in sorted_data]

    # 6번 : prod_list에서 상위 5개의 제품을 출력한다
    result = pd.DataFrame(prod_list[:5], columns= [f'나와 비슷한 사용자가 만족한 {product_type} 제품'])
    print(result)


In [68]:
user_based_recommend(1, '토너')

  나와 비슷한 사용자가 만족한 토너 제품
0               toner_6
1               toner_2
2              toner_11
3               toner_8
4               toner_1
